In [6]:
import tensorflow as tf
import numpy
import os, sys, pathlib

from omegaconf import OmegaConf

from mlqm import DEFAULT_TENSOR_TYPE
tf.keras.backend.set_floatx(DEFAULT_TENSOR_TYPE)

# For the wavefunction:
from mlqm.models import ManyBodyWavefunction
from mlqm.config import ManyBodyCfg

# For the hamiltonian:
from mlqm.hamiltonians import Hamiltonian
from mlqm.config import NuclearHamiltonian as H_config


In [7]:

def generate_inputs(nwalkers, nparticles, ndim, n_spin_up, n_protons):

    inputs = numpy.random.uniform(size=[nwalkers, nparticles, ndim])


    # Note that we initialize with NUMPY for ease of indexing and shuffling
    spin_walkers = numpy.zeros(shape=(nwalkers, nparticles)) - 1
    for i in range(n_spin_up):
        if i < spin_walkers.shape[-1]:
            spin_walkers[:,i] += 2

    # Shuffle the spin up particles on each axis:

    # How to compute many permutations at once?
    #  Answer from https://stackoverflow.com/questions/5040797/shuffling-numpy-array-along-a-given-axis
    # Bottom line: gen random numbers for each axis, sort only in that axis,
    # and apply the permutations
    idx = numpy.random.rand(*spin_walkers.shape).argsort(axis=1)
    spin_walkers = numpy.take_along_axis(spin_walkers, idx, axis=1)

    # Note that we initialize with NUMPY for ease of indexing and shuffling
    isospin_walkers = numpy.zeros(shape=(nwalkers, nparticles)) - 1
    for i in range(n_protons):
        if i < isospin_walkers.shape[-1]:
            isospin_walkers[:,i] += 2

    # Shuffle the spin up particles on each axis:

    # How to compute many permutations at once?
    #  Answer from https://stackoverflow.com/questions/5040797/shuffling-numpy-array-along-a-given-axis
    # Bottom line: gen random numbers for each axis, sort only in that axis,
    # and apply the permutations
    idx = numpy.random.rand(*isospin_walkers.shape).argsort(axis=1)
    isospin_walkers = numpy.take_along_axis(isospin_walkers, idx, axis=1)

    return inputs, spin_walkers, isospin_walkers

In [34]:
def compute_derivatives(
    wavefunction : tf.keras.models.Model,
    inputs : tf.Tensor,
    spin : tf.Tensor,
    isospin : tf.Tensor = None):

    n_walkers = inputs.shape[0]
    n_particles = inputs.shape[1]
    n_dim = inputs.shape[2]

    # Turning off all tape watching except for the inputs:
    # Using the outer-most tape to watch the computation of the first derivative:
    with tf.GradientTape() as tape:
        # Use the inner tape to watch the computation of the wavefunction:
        tape.watch(inputs)
        with tf.GradientTape() as second_tape:
            second_tape.watch(inputs)
            w_of_x = wavefunction(inputs, spin, isospin)
            # w_of_x = tf.reshape(sign, (-1, 1)) * tf.exp(logw_of_x)
        # Get the derivative of logw_of_x with respect to inputs
        print(w_of_x)
        dw_dx = second_tape.gradient(w_of_x, inputs)
#         dw_dx = tf.reshape(dw_dx, (n_walkers, n_particles, n_dim))
#         print(dw_dx)

    # Get the derivative of dlogw_dx with respect to inputs (aka second derivative)

    # We have to extract the diagonal of the jacobian, which comes out with shape
    # [nwalkers, nparticles, dimension, nwalkers, nparticles, dimension]

    # The indexes represent partial derivative indexes, so,
    # d2w_dx2[i_w, n1,d1, n2, d2] represents the second derivative of the
    # wavefunction at dimension d1

    # This is the full hessian computation:
    d2w_dx2 = tape.batch_jacobian(dw_dx, inputs)

    print(d2w_dx2)

    # # Extract the diagonal parts:
    # d2w_dx2 = tf.vectorized_map(tf.linalg.tensor_diag_part, d2w_dx2)

    # # And this contracts:
    d2w_dx2 = tf.einsum("wpdpd->wpd",d2w_dx2)
    #
    # print("First einsum: ", d2w_dx2[0])
    #
    # print("Method diff 0: ", d2w_dx2_t[0] - d2w_dx2[0])
    # # TODO: test that the second derivative is correct with finite differences

    return w_of_x, dw_dx, d2w_dx2

In [9]:
nwalkers   = 1
nparticles = 2
ndim       = 1

In [24]:
inputs, spins, isospins = generate_inputs(nwalkers, nparticles, ndim, 2, 1)
xinputs = tf.convert_to_tensor(
    inputs - numpy.reshape(numpy.mean(inputs, axis=1), (nwalkers, 1, ndim)))


In [25]:
print(isospins)

[[-1.  1.]]


In [26]:
 c = ManyBodyCfg()

c = OmegaConf.structured(c)
w = ManyBodyWavefunction(ndim, nparticles, c,
    n_spin_up = 2, n_protons = 1,
    use_spin = True, use_isospin = True
)


In [35]:
w_local = w(xinputs, spins, isospins)


w_of_x, dw_dx, d2w_dx2 = compute_derivatives(w, xinputs, spins, isospins)

dw_dx = dw_dx.numpy()
d2w_dx2 = d2w_dx2.numpy()


tf.Tensor([[0.00074885]], shape=(1, 1), dtype=float64)
tf.Tensor(
[[[[[ 0.01066629]
    [-0.01066629]]]


  [[[-0.01066629]
    [ 0.01066629]]]]], shape=(1, 2, 1, 2, 1), dtype=float64)


In [36]:
w_of_x

<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[0.00074885]])>

In [37]:
w_of_x - w_local

<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[0.]])>

In [38]:
def compute_numerical_derivatives(f, x, s, i, dim, part, kick_size=1e-4):
    # Get the shapes:
    nwalkers = x.shape[0]
    nparticles = x.shape[1]
    # Placeholder for a kick:
    kick = numpy.zeros(shape = x.shape)
    kick_size = 1e-4
    
    walkers = numpy.arange(nwalkers)

    if len(kick.shape) == 3:
        # Not single-particle
        kick[walkers,part, dim] += kick_size
    elif len(kick.shape) == 2:
        # single particle:
        kick[walkers, dim] += kick_size

#     print(kick)
    
    # x + dx:
    kicked_up_input = x + \
            tf.convert_to_tensor(kick, dtype=DEFAULT_TENSOR_TYPE)

#     # x + 2*dx:
#     kicked_double_up_input = x + \
#         tf.convert_to_tensor(2*kick, dtype=DEFAULT_TENSOR_TYPE)
    
    # x - dx
    kicked_down_input = x - \
        tf.convert_to_tensor(kick, dtype=DEFAULT_TENSOR_TYPE)

#     # x - 2*dx
#     kicked_double_down_input = x - \
#         tf.convert_to_tensor(2*kick, dtype=DEFAULT_TENSOR_TYPE)
    
    central_value = f(x, s, i)
    w_up = f(kicked_up_input, s, i)
    w_down = f(kicked_down_input, s, i)
#     w_up_up = f(kicked_double_up_input, s, i)
#     w_down_down = f(kicked_double_down_input, s, i)

    
    # Use numpy to make slicing easier
    w_prime_fd = tf.reshape((w_up - w_down) / (2*kick_size), (nwalkers,)).numpy()
    # What about the second derivative?

    # https://math.stackexchange.com/questions/3756717/finite-differences-second-derivative-as-successive-application-of-the-first-deri
    # This gives precision of O(kick**4)
#     w_prime_prime_num = -w_down_down + 16*w_down - 30* w_of_x + 16 * w_up - w_up_up
    w_prime_prime_num = w_up + w_down - 2*central_value
    w_prime_prime_fd = tf.reshape(w_prime_prime_num/ (kick_size**2), (nwalkers,)).numpy()

    return w_prime_fd, w_prime_prime_fd

In [42]:

first = []
second = []
for dim in range(ndim):
    first.append([])
    second.append([])
    for part in range(nparticles):

        t_num_dw_dx, t_num_d2w_dx2 = compute_numerical_derivatives(w, xinputs, spins, isospins, dim, part, kick_size=1e-6)
        first[dim].append(t_num_dw_dx)
        second[dim].append(t_num_d2w_dx2)
    # At the end of the loop, the list should be length n_particles, with nwalker entries each.
    # stack and flip it
    first[-1] = numpy.stack(first[-1]).T
    second[-1] = numpy.stack(second[-1]).T

num_dw_dx = numpy.stack(first, axis=-1)
num_d2w_dx2 = numpy.stack(second, axis=-1)
print(num_dw_dx)
print(num_d2w_dx2)

[[[-0.00238138]
  [ 0.00238138]]]
[[[0.00195842]
  [0.00195842]]]


In [40]:
num_dw_dx - dw_dx

array([[[ 1.18258155e-11],
        [-1.18258155e-11]]])

In [41]:
num_d2w_dx2 - d2w_dx2

array([[[-0.00870786],
        [-0.00870786]]])

In [254]:
o_l_of_x, o_dl_dx, o_d2l_dx2 = h.compute_derivatives(l, xinputs, spins, other_isospins)
o_fd_dl_dx, o_fd_d2l_dx2 = compute_numerical_derivatives(l, xinputs, spins, other_isospins,part,dim)

In [255]:
o_fd_dl_dx - o_dl_dx[:,part,dim]

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([3.11348447e-11, 2.85769741e-11, 3.43116646e-11])>

In [256]:
o_fd_d2l_dx2 - o_d2l_dx2[:,part,dim]

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 6.36841540e-10,  2.49571932e-09, -3.08548720e-09])>

# Just the correlator

In [35]:
c = lambda x, s, i, training=False: tf.math.exp(w.correlator(x))

In [36]:
c_of_x, dc_dx, d2c_dx2 = h.compute_derivatives(c, xinputs, spins, isospins)


In [37]:
dim = 2; part=1
fd_dc_dx, fd_d2c_dx2 = compute_numerical_derivatives(c, xinputs, spins, isospins, dim, part)

In [38]:
dc_dx[:,part,dim] - fd_dc_dx  

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([-1.37020617e-09, -6.43131895e-10, -9.71684150e-10])>

In [39]:
d2c_dx2[:,part,dim] - fd_d2c_dx2

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([-2.82540900e-08,  1.42938258e-08,  1.92497839e-08])>

In [41]:
# Check all the coordinates:
c_of_x, dc_dx, d2c_dx2 = h.compute_derivatives(c, xinputs, spins, isospins)

for dim in range(ndim):
    for part in range(nparticles):
        fd_dc_dx, fd_d2c_dx2 = compute_numerical_derivatives(c, xinputs, spins, isospins, dim, part) 
        assert (dc_dx[:,part,dim] - fd_dc_dx  < 1e-4).numpy().all()
        assert (d2c_dx2[:,part,dim] - fd_d2c_dx2 < 1e-4).numpy().all()

## The correlator works!

# Just a single spatial net:

In [42]:
sn = c = lambda x, s, i, training=False: w.spatial_nets[0](x)


In [43]:
sn_of_x, dsn_dx, d2sn_dx2 = h.compute_derivatives(sn, xinputs, spins, isospins)
print(sn_of_x)

tf.Tensor(
[[[ 0.03046872]
  [ 0.12013633]]

 [[-0.00304384]
  [ 0.14237937]]

 [[ 0.06376086]
  [ 0.09589538]]], shape=(3, 2, 1), dtype=float64)


In [44]:
dim = 2; part=1
fd_dsn_dx, fd_d2sn_dx2 = compute_numerical_derivatives(sn, xinputs[:,part,:], spins, isospins, dim, part)

In [45]:
dsn_dx[:,part,dim] - fd_dsn_dx

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([4.51136142e-11, 1.25718186e-11, 8.75603617e-11])>

In [46]:
d2sn_dx2[:,part,dim] - fd_d2sn_dx2

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([-1.42281443e-08, -1.75155959e-08, -6.36823973e-09])>

In [48]:
# Check all the coordinates:

for net_index in range(nparticles):
    sn = c = lambda x, s, i, training=False: w.spatial_nets[net_index](x)
    sn_of_x, dsn_dx, d2sn_dx2 = h.compute_derivatives(sn, xinputs, spins, isospins)
    for dim in range(ndim):
        for part in range(nparticles):
            fd_dsn_dx, fd_d2sn_dx2 = compute_numerical_derivatives(sn, xinputs[:,part,:], spins, isospins, dim, part)
            assert(fd_dsn_dx.shape[0] == nwalkers)
            assert (dsn_dx[:,part,dim] - fd_dsn_dx < 1e-4).numpy().all()
            assert (d2sn_dx2[:,part,dim] - fd_d2sn_dx2 < 1e-4).numpy().all()       

In [49]:
print(fd_dsn_dx.shape)

(3,)


## Both spatial nets works!

What about the vectorized spatial net implementation?

In [51]:
w.compute_row(w.spatial_nets[0], xinputs)

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[ 0.03046872,  0.12013633],
       [-0.00304384,  0.14237937],
       [ 0.06376086,  0.09589538]])>

In [53]:
tf.reshape(w.spatial_nets[0](xinputs), (nwalkers, nparticles))

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[ 0.03046872,  0.12013633],
       [-0.00304384,  0.14237937],
       [ 0.06376086,  0.09589538]])>

In [54]:
tf.reshape(w.spatial_nets[1](xinputs), (nwalkers, nparticles))

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[-0.51251004,  0.48281624],
       [-0.24184365,  0.18608135],
       [-0.00431047, -0.04023421]])>

In [55]:
w.compute_spatial_slater(xinputs)

<tf.Tensor: shape=(3, 2, 2), dtype=float64, numpy=
array([[[ 0.03046872,  0.12013633],
        [-0.51251004,  0.48281624]],

       [[-0.00304384,  0.14237937],
        [-0.24184365,  0.18608135]],

       [[ 0.06376086,  0.09589538],
        [-0.00431047, -0.04023421]]])>

In [56]:
tf.linalg.det(w.compute_spatial_slater(xinputs))

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 0.07628186,  0.03386714, -0.00215201])>

# What about just the coordinate parts?

In [57]:
r = lambda x, s, i, training=False: tf.linalg.det(w.compute_spatial_slater(x))

In [58]:
r_of_x, o_dr_dx, o_d2r_dx2 = h.compute_derivatives(r, xinputs, spins, isospins)


In [59]:
r_of_x

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 0.07628186,  0.03386714, -0.00215201])>

In [60]:
dim = 2; part=0
fd_dr_dx, fd_d2r_dx2 = compute_numerical_derivatives(r, xinputs, spins, isospins, dim, part)

In [61]:
o_dr_dx[:,part,dim] - fd_dr_dx

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([-3.20553251e-10, -6.32720015e-10, -4.65971040e-10])>

In [62]:
o_d2r_dx2[:,part,dim] - fd_d2r_dx2

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ -0.24846157,   1.53342546, -15.29529662])>

# Isolating the 2nd derivative of the determinants.

It sure seems like the major issue is that the second derivative of a determinant is not working.  Let's demonstrate it with a simpler example:

In [196]:
# Create a low-level function for each row:
def f(this_input, _alpha):
   return tf.exp(- tf.reduce_sum(_alpha * this_input**2, axis=(2)))

In [197]:
net_1 = lambda _i : f(_i, 11)
net_2 = lambda _i : f(_i, 7.4)

In [198]:
def compute_matrix(_x, n1, n2):
    row_1 = n1(_x) # Shape should be (nwalkers, nparticles)
    row_2 = n2(_x)
    matrix = tf.stack([row_1, row_2], axis=1)
    return matrix

In [199]:
compute_matrix(inputs, net_1, net_2)

<tf.Tensor: shape=(3, 2, 2), dtype=float64, numpy=
array([[[4.43496775e-01, 1.91415902e-04],
        [5.78700139e-01, 3.15344463e-03]],

       [[3.59072265e-03, 1.33788020e-06],
        [2.26623092e-02, 1.11861510e-04]],

       [[1.65738205e-03, 1.22184987e-03],
        [1.34719285e-02, 1.09738106e-02]]])>

In [145]:
detmat = lambda x, s=None, i=None, training=False : tf.linalg.det(compute_matrix(x, net_1, net_2))

In [146]:
detmat(inputs)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([1.28777011e-03, 3.71344205e-07, 1.72712259e-06])>

In [147]:
f(xinputs, 0.1).shape

TensorShape([3, 2])

In [151]:
detmat(inputs)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([1.28777011e-03, 3.71344205e-07, 1.72712259e-06])>

In [160]:
d_of_x, , d2d_dx2 = h.compute_derivatives(detmat, inputs, spins, isospins)

In [154]:
d_of_x

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([1.28777011e-03, 3.71344205e-07, 1.72712259e-06])>

In [157]:
dim=0; part=0
fd_dd_dx, fd_d2d_dx2 = compute_numerical_derivatives(detmat, inputs, spins, isospins, dim, part)

In [158]:
dd_dx[:,part, dim] - fd_dd_dx 

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([-3.21546831e-10,  9.34807664e-13,  7.32400504e-11])>

In [227]:
d2d_dx2[:,part, dim] - fd_d2d_dx2 

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([0.45036956, 0.02123967, 0.09771254])>

The second derivative is not matching.  But it can be computed numerically.

# What about pytorch?

In [162]:
import torch

In [170]:
# Create a low-level function for each row:
def f_torch(this_input, _alpha):
   return torch.exp(- torch.sum(_alpha * this_input**2, dim=2))

In [180]:
f_torch(torch.tensor(inputs), 0.1)

tensor([[0.9926, 0.9251],
        [0.9501, 0.8843],
        [0.9435, 0.9408]], dtype=torch.float64)

In [181]:
f(inputs, 0.1)

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[0.99263575, 0.92512369],
       [0.95011108, 0.88430849],
       [0.94345681, 0.94084557]])>

In [191]:
net_1_torch = lambda _i : f_torch(_i, 11)
net_2_torch = lambda _i : f_torch(_i, 7.4)

In [192]:
def compute_matrix_torch(_x, n1, n2):
    row_1 = n1(_x) # Shape should be (nwalkers, nparticles)
    row_2 = n2(_x)
    matrix = torch.stack([row_1, row_2], axis=1)
    return matrix

In [194]:
compute_matrix_torch(torch.tensor(inputs), net_1_torch, net_2_torch)

tensor([[[4.4350e-01, 1.9142e-04],
         [5.7870e-01, 3.1534e-03]],

        [[3.5907e-03, 1.3379e-06],
         [2.2662e-02, 1.1186e-04]],

        [[1.6574e-03, 1.2218e-03],
         [1.3472e-02, 1.0974e-02]]], dtype=torch.float64)

In [200]:
compute_matrix(inputs, net_1, net_2)

<tf.Tensor: shape=(3, 2, 2), dtype=float64, numpy=
array([[[4.43496775e-01, 1.91415902e-04],
        [5.78700139e-01, 3.15344463e-03]],

       [[3.59072265e-03, 1.33788020e-06],
        [2.26623092e-02, 1.11861510e-04]],

       [[1.65738205e-03, 1.22184987e-03],
        [1.34719285e-02, 1.09738106e-02]]])>

In [214]:
detmat_torch = lambda x, s=None, i=None, training=False : torch.linalg.det(compute_matrix_torch(x, net_1_torch, net_2_torch))

In [215]:
torch_inputs = torch.tensor(inputs, requires_grad=True)
output = detmat_torch(torch_inputs)

In [216]:
output

tensor([1.2878e-03, 3.7134e-07, 1.7271e-06], dtype=torch.float64,
       grad_fn=<LinalgDetBackward>)

In [220]:
first_grad = torch.autograd.grad(output, torch_inputs)

RuntimeError: grad can be implicitly created only for scalar outputs

In [218]:
torch_inputs.grad

tensor([[[-3.1415e-03, -7.0255e-03, -1.8675e-03],
         [-6.5121e-05, -1.4066e-02, -7.8540e-03]],

        [[-5.0248e-06, -1.2191e-07, -3.2776e-06],
         [-3.5667e-06, -4.5528e-06, -8.9185e-07]],

        [[-1.1481e-04, -1.9242e-05, -2.6573e-05],
         [ 6.4056e-05,  3.3989e-05,  3.2202e-06]]], dtype=torch.float64)

In [219]:
dd_dx

<tf.Tensor: shape=(3, 2, 3), dtype=float64, numpy=
array([[[-3.14152998e-03, -7.02550577e-03, -1.86753293e-03],
        [-6.51206659e-05, -1.40659282e-02, -7.85402683e-03]],

       [[-5.02479899e-06, -1.21913067e-07, -3.27755920e-06],
        [-3.56668779e-06, -4.55284709e-06, -8.91848135e-07]],

       [[-1.14811823e-04, -1.92416681e-05, -2.65727410e-05],
        [ 6.40557560e-05,  3.39893400e-05,  3.22016415e-06]]])>

In [226]:
torch.autograd.functional.hessian(detmat_torch, torch_inputs)

RuntimeError: The Tensor returned by the function given to hessian should contain a single element